**Install Packages**

In [1]:
from google.colab import output, drive, files # specific to Google Colab
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import warnings

# settings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

**Functions**

In [3]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

**Locals & Constants**

In [4]:
############
# OPTIONAL #
############

# mount drive
#drive.mount("/content/drive", force_remount=False)

# data location
file_dir = "/content/drive/My Drive/Colab Data/input/" # optional



# New Section

In [5]:
# read in api key file
#df_api_keys = pd.read_csv(file_dir + "api_keys.csv")
# get keys
#api_key = df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key
api_key ='156b46f9-1d41-40b6-8052-62510a6d3327'

**Data**

**1. Property Listings For Sale**

In [6]:
# zillow search url
listing_url = "https://www.zillow.com/tampa-fl/houses/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.96171153027345%2C%22east%22%3A-81.94684946972657%2C%22south%22%3A27.709897826482354%2C%22north%22%3A28.2786076346999%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41176%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A200000%2C%22max%22%3A550000%7D%2C%22mp%22%3A%7B%22min%22%3A946%2C%22max%22%3A2366%7D%2C%22beds%22%3A%7B%22min%22%3A2%7D%2C%22built%22%3A%7B%22min%22%3A1940%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D"

# get listings
listing_response = get_listings(api_key, listing_url)

In [7]:
# view all keys
listing_response.json().keys()

dict_keys(['is_success', 'data', 'message'])

In [8]:
# check if request is successful
listing_response.json()["is_success"]

True

In [9]:
# view count of properies returned in request
num_of_properties = listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 252


In [10]:
# view all listings
df_listings = pd.json_normalize(listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_listings))
print("Number of columns:", len(df_listings.columns))
df_listings

Number of rows: 250
Number of columns: 80


zpid     price priceLabel  beds  baths  area statusType  \
0     82999927  $433,000      $433K     4    2.0  1739   FOR_SALE   
1     45037036  $464,900      $465K     4    2.0  1986   FOR_SALE   
2    154856727  $349,900      $350K     3    2.0  1644   FOR_SALE   
3     45040725  $335,000      $335K     3    2.0   836   FOR_SALE   
4    335587788  $480,000      $480K     3    2.0  2076   FOR_SALE   
..         ...       ...        ...   ...    ...   ...        ...   
245   45098095  $320,000      $320K     3    1.0   916   FOR_SALE   
246   45036689  $305,000      $305K     2    1.0   938   FOR_SALE   
247   45082828  $349,900      $350K     3    1.0  1217   FOR_SALE   
248   44928986  $299,000      $299K     2    1.0   738   FOR_SALE   
249   45072670  $520,000      $520K     2    1.0   640   FOR_SALE   

           statusText  isFavorite  isUserClaimingOwner  isUserConfirmedClaim  \
0      House for sale       False                False                 False   
1      House for sale       False                False                 False   
2    New construction       False                False                 False   
3      House for sale       False                False                 False   
4      House for sale       False                False                 False   
..                ...         ...                  ...                   ...   
245    House for sale       False                False                 False   
246    House for sale       False                False                 False   
247    House for sale       False                False                 False   
248    House for sale       False                False                 False   
249    House for sale       False                False                 False   

                                                imgSrc  hasImage  visited  \
0    https://photos.zillowstatic.com/fp/fb302fdecf1...      True    False   
1    https://photos.zillowstatic.com/fp/2604fe8c389...      True    False   
2    https://photos.zillowstatic.com/fp/b730d9aae8f...      True    False   
3    https://photos.zillowstatic.com/fp/4fded623e6d...      True    False   
4    https://photos.zillowstatic.com/fp/4d95f68c3c5...      True    False   
..                                                 ...       ...      ...   
245  https://photos.zillowstatic.com/fp/6fede61d98f...      True    False   
246  https://photos.zillowstatic.com/fp/2e0787edf47...      True    False   
247  https://photos.zillowstatic.com/fp/acb5578a2d1...      True    False   
248  https://photos.zillowstatic.com/fp/c42c6776611...      True    False   
249  https://photos.zillowstatic.com/fp/f7929ea8f19...      True    False   

          listingType  shouldShowZestimateAsPrice  \
0                                           False   
1                                           False   
2    NEW_CONSTRUCTION                       False   
3                                           False   
4                                           False   
..                ...                         ...   
245                                         False   
246                                         False   
247                                         False   
248                                         False   
249                                         False   

                                             detailUrl    pgapt  \
0    /homedetails/10609-Liberty-Bell-Dr-Tampa-FL-33...  ForSale   
1    /homedetails/801-W-Yukon-St-Tampa-FL-33604/450...  ForSale   
2    /homedetails/3305-E-Chelsea-St-Tampa-FL-33610/...  ForSale   
3    /homedetails/1742-W-Comanche-Ave-Tampa-FL-3360...  ForSale   
4    /homedetails/16112-Monterey-Greens-Cir-Tampa-F...  ForSale   
..                                                 ...      ...   
245  /homedetails/5203-N-9th-St-Tampa-FL-33603/4509...  ForSale   
246  /homedetails/8705-N-Highland-Ave-Tampa-FL-3360...  ForSale   
247  /homedetails/1515-E-Hanna-Ave-Tampa-FL-33610/4... 

In [11]:
df_listings.to_csv("df_listings.csv")
files.download("df_listings.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# view price
px.histogram(df_listings, x="hdpData.homeInfo.price", title="Sales Price Histogram")

In [ ]:
# view zestimate
px.histogram(df_listings, x="hdpData.homeInfo.zestimate", title="Zestimate Histogram")



In [ ]:
# view rent zestimate
px.box(df_listings, x="hdpData.homeInfo.rentZestimate", title="Rent Zestimate Box Plot")
